In [1]:
import pandas as pd

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

driver = webdriver.Chrome("../../Data Science/chromedriver")

<ipython-input-3-b030b2f9e5e3>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("../../Data Science/chromedriver")


In [4]:
wa_cities_page = "https://www.apartments.com/sitemap/washington/cities/"
driver.get(wa_cities_page)

In [5]:
hrefs = driver.find_elements(By.XPATH,"//a[@href]")
wa_city_links = []
for x in hrefs:
    link = x.get_attribute("href")
    if link.endswith("-wa/"):
        wa_city_links.append(link)
wa_city_links[:10]

['https://www.apartments.com/seattle-wa/',
 'https://www.apartments.com/aberdeen-wa/',
 'https://www.apartments.com/airway-heights-wa/',
 'https://www.apartments.com/allyn-wa/',
 'https://www.apartments.com/amboy-wa/',
 'https://www.apartments.com/anacortes-wa/',
 'https://www.apartments.com/arlington-wa/',
 'https://www.apartments.com/auburn-wa/',
 'https://www.apartments.com/bainbridge-island-wa/',
 'https://www.apartments.com/battle-ground-wa/']

In [6]:
def get_single_property_info(driver,link):
        driver.get(link)
        try: built_date = driver.find_element(By.XPATH,"//div[contains( text( ), 'Built in')]").text
        except: built_date = None
        try: 
            unit_counts = driver.find_elements(By.XPATH,"//div[contains( text( ), 'unit')]")
            unit_count = [x.text for x in unit_counts if "available" not in x.text.lower() and 'unit' in x.text.lower()][0]
        except:
            unit_count = None
        property_name = driver.find_element(By.ID,'propertyName').text
        try:
            address_elem = driver.find_elements(By.CLASS_NAME,"propertyAddressContainer")
            address,neighborhood = address_elem[0].text.split("\n")
        except:
            neighborhood = driver.find_element(By.CLASS_NAME,'neighborhood').text
            address = property_name
        single_property_info = {'built_date':built_date,'unit_count':unit_count,'address':address,
                                'neighborhood':neighborhood,'property_name':property_name}
        return(single_property_info)

In [ ]:
all_property_info = dict()
# Loop through cities
for city_link in wa_city_links:
    # Apartments/condos only
    city_link = city_link.replace("apartments.com/","apartments.com/apartments-condos/")
    driver.get(city_link)
    # Check for no results
    try:
        no_results_text = driver.find_element(By.CLASS_NAME,"no-results").text
        if "NO RESULTS FOUND" in no_results_text:
            print("No results found",city_link)
            continue
    except:
        pass
    # Get count of pages for each city
    try:
        page_range = driver.find_element(By.CLASS_NAME,"pageRange").text
        n_pages = int(page_range.split()[-1])
    except:
        n_pages = 1
    all_property_links = []
    # Loop through pages of listings to get property links
    for x in range(1,n_pages+1):
        driver.get(f"{city_link}{x}/")
        link_elements = driver.find_elements(By.CLASS_NAME,'property-link')
        property_links = [x.get_attribute("href") for x in link_elements]
        all_property_links.extend(property_links)
    # Loop through properties
    for x in all_property_links:
        all_property_info[x] = get_single_property_info(driver,x)

In [ ]:
df_wa_property_info = pd.DataFrame(all_property_info).T.reset_index(names="url")
df_wa_property_info.head()

# Single City Testing

In [119]:
base_url = "http://apartments.com/seattle-wa"
driver.get(base_url)

In [26]:
#search_bar = driver.find_element(By.ID,"quickSearchLookup")

In [125]:
page_range = driver.find_element(By.CLASS_NAME,"pageRange").text
n_pages = int(page_range.split()[-1])

28

In [90]:
all_links = []
for x in range(1,n_pages+1):
    driver.get(f"{base_url}/{x}/")
    link_elements = driver.find_elements(By.CLASS_NAME,'property-link')
    links = [x.get_attribute("href") for x in link_elements]
    all_links.extend(links)

In [92]:
built_date = driver.find_element(By.XPATH,"//div[contains( text( ), 'Built in')]").text
built_date

'Built in 2022'

In [107]:
unit_counts = driver.find_elements(By.XPATH,"//div[contains( text( ), 'unit')]")
unit_count = [x.text for x in unit_counts if "available" not in x.text.lower() and 'unit' in x.text.lower()][0]
unit_count

'118 units/5 stories'

In [118]:
address_elem = driver.find_elements(By.CLASS_NAME,"propertyAddressContainer")
address,neighborhood = address_elem[0].text.split("\n")
print(address)
print(neighborhood)

4106 N Stone Way, Seattle, WA 98103
Wallingford


In [44]:
#driver.find_element(By.CLASS_NAME,'propertyName').get_attribute("innerHTML")

'\n                    <span id="mediaGalleryModalLabel">\n                        171 E Allynview Dr <span class="screenReaderOnly">media gallery</span>\n                    </span>\n                    <span id="mediaGalleryModalLabelEngrainUnitLabel"><span> Unit</span><span id="engrainUnitId"></span></span>\n                '